In [1]:
import numpy as np
import sympy as sym
from scipy import linalg
from scipy import signal

# Suppress the use of scientific notation when printing small numbers
np.set_printoptions(suppress=True)

# Is the reaction wheel controllable?

Parameters:

In [2]:
# radius of wheel
rw = 0.5

# mass of wheel
mw = 0.25

# moi of wheel
Jw = 0.5 * mw * rw**2

Equations of motion:

$$ J_w \dot{v} = \tau $$

In [3]:
# Convert floats to rationals
Jw_sym = sym.nsimplify(Jw)

# Create symbolic variables
v, tau = sym.symbols('v, tau', real=True)

# Define equations of motion
f = sym.Matrix([[(1 / Jw_sym) * tau]])

# Show the result
f

Matrix([[32*tau]])

Linearization:

In [4]:
ve = 1. * (2 * np.pi) # <-- 1 rpm
taue = 0.

A_num = sym.lambdify((v, tau), f.jacobian([v]))
B_num = sym.lambdify((v, tau), f.jacobian([tau]))
A = A_num(ve, taue).astype(float)
B = B_num(ve, taue).astype(float)

In [5]:
print(A)

[[0.]]


In [6]:
print(B)

[[32.]]


Controllability matrix:

In [7]:
W = np.block([[B]])

In [8]:
print(W)

[[32.]]


Rank of controllability matrix:

In [9]:
np.linalg.matrix_rank(W)

1

Number of states:

In [10]:
A.shape[0]

1

What happens if you try to do eigenvalue placement?

In [11]:
fbk = signal.place_poles(A, B, [-1.])
K = fbk.gain_matrix

Did it work?

In [12]:
print(linalg.eigvals(A - B @ K))

[-1.+0.j]


# Is the platform controllable, ignoring the reaction wheel?

Parameters:

In [14]:
# distance from platform axis to wheel axis
l = 1.

# radius of wheel
rw = 0.5

# mass of wheel
mw = 0.25

# moi of wheel
Jw = 0.5 * mw * rw**2

# mass of platform
mp = 12. * (1. - mw * l**2) / (3.**2 + 2.**2)

# moment of inertia of platform
Jp = (1. / 12.) * mp * (3.**2 + 2.**2)

Equations of motion:

$$ (J_p + ml^2) \ddot{q} = \tau $$

In [15]:
# Convert floats to rationals
l_sym = sym.nsimplify(l)
mw_sym = sym.nsimplify(mw)
Jp_sym = sym.nsimplify(Jp)

# Create symbolic variables
q, v, tau = sym.symbols('q, v, tau', real=True)

# Define equations of motion
f = sym.Matrix([[v], [(1 / (Jp_sym + mw_sym * l_sym**2)) * tau]])

# Show the result
f

Matrix([
[  v],
[tau]])

Linearization:

In [16]:
qe = 0.
ve = 0.
taue = 0.

A_num = sym.lambdify((q, v, tau), f.jacobian([q, v]))
B_num = sym.lambdify((q, v, tau), f.jacobian([tau]))
A = A_num(qe, ve, taue).astype(float)
B = B_num(qe, ve, taue).astype(float)

In [17]:
print(A)

[[0. 1.]
 [0. 0.]]


In [18]:
print(B)

[[0.]
 [1.]]


Controllability matrix:

In [19]:
W = np.block([[B, A @ B]])

In [20]:
print(W)

[[0. 1.]
 [1. 0.]]


Rank of controllability matrix:

In [21]:
np.linalg.matrix_rank(W)

2

Number of states:

In [22]:
A.shape[0]

2

What happens if you try to do eigenvalue placement?

In [27]:
fbk = signal.place_poles(A, B, [-1., -2.])
K = fbk.gain_matrix

Did it work?

In [28]:
print(linalg.eigvals(A - B @ K))

[-1.+0.j -2.+0.j]


In [29]:
print(K.tolist())

[[2.0000000000000004, 3.0]]


# Is the platform controllable, *not* ignoring the reaction wheel?

Parameters:

In [30]:
# distance from platform axis to wheel axis
l = 1.

# radius of wheel
rw = 0.5

# mass of wheel
mw = 0.25

# moi of wheel
Jw = 0.5 * mw * rw**2

# mass of platform
mp = 12. * (1. - mw * l**2) / (3.**2 + 2.**2)

# moment of inertia of platform
Jp = (1. / 12.) * mp * (3.**2 + 2.**2)

Equations of motion:

$$
\begin{aligned}
J_1 \ddot{q}_1 &= \tau \\
J_2 \dot{v}_2 &= - \left( \dfrac{J_1 + J_2}{J_1} \right) \tau
\end{aligned}
$$

where

$$
J_1 = J_p + m_wl^2
\qquad
\qquad
J_2 = J_w
$$

In [31]:
# Convert floats to rationals
J1 = sym.nsimplify(Jp + mw * l**2)
J2 = sym.nsimplify(Jw)

# Create symbolic variables
q1, v1, v2, tau = sym.symbols('q1, v1, v2, tau', real=True)

# Define equations of motion
f = sym.Matrix([[v1], [(1 / J1) * tau], [(1 / J2) * (- ((J1 + J2) / J1) * tau)]])

# Show the result
f

Matrix([
[     v1],
[    tau],
[-33*tau]])

Linearization:

In [32]:
q1e = 0.
v1e = 0.
v2e = 0.
taue = 0.

A_num = sym.lambdify((q1, v1, v2, tau), f.jacobian([q1, v1, v2]))
B_num = sym.lambdify((q1, v1, v2, tau), f.jacobian([tau]))
A = A_num(q1e, v1e, v2e, taue).astype(float)
B = B_num(q1e, v1e, v2e, taue).astype(float)

In [33]:
print(A)

[[0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [34]:
print(B)

[[  0.]
 [  1.]
 [-33.]]


Controllability matrix:

In [35]:
W = np.block([[B, A @ B, A @ A @ B]])

In [36]:
print(W)

[[  0.   1.   0.]
 [  1.   0.   0.]
 [-33.   0.   0.]]


Rank of controllability matrix:

In [37]:
np.linalg.matrix_rank(W)

2

Number of states:

In [38]:
A.shape[0]

3

In [39]:
linalg.inv(W)

LinAlgError: singular matrix

What happens if you try to do eigenvalue placement?

In [41]:
fbk = signal.place_poles(A, B, [-1., -2., -10.])
K = fbk.gain_matrix

Did it work?

In [42]:
print(linalg.eigvals(A - B @ K))

[ 0.00000000e+00+0.j -3.85400528e+08+0.j  3.85400528e+08+0.j]


# What if there is damping?

Parameters:

In [43]:
# distance from platform axis to wheel axis
l = 1.

# radius of wheel
rw = 0.5

# mass of wheel
mw = 0.25

# moi of wheel
Jw = 0.5 * mw * rw**2

# mass of platform
mp = 12. * (1. - mw * l**2) / (3.**2 + 2.**2)

# moment of inertia of platform
Jp = (1. / 12.) * mp * (3.**2 + 2.**2)

# coefficient of viscous friction
c1 = 0.1
c2 = 0.1

Equations of motion:

$$
\begin{aligned}
J_1 \ddot{q}_1 &= \tau - c_1 v_1 + c_2 v_2 \\
J_2 \dot{v}_2 &= - \left( \dfrac{J_1 + J_2}{J_1} \right) \tau + c_1 \left(\dfrac{J_2}{J_1}\right) v_1 - c_2 \left( \dfrac{J_1 + J_2}{J_1} \right) v_2
\end{aligned}
$$

where

$$
J_1 = J_p + m_wl^2
\qquad
\qquad
J_2 = J_w
$$

In [44]:
# Convert floats to rationals
J1 = sym.nsimplify(Jp + mw * l**2)
J2 = sym.nsimplify(Jw)
c1 = sym.nsimplify(c1)
c2 = sym.nsimplify(c2)

# Create symbolic variables
q1, v1, v2, tau = sym.symbols('q1, v1, v2, tau', real=True)

# Define equations of motion
f = sym.Matrix([[v1],
                [(1 / J1) * (tau - c1 * v1 + c2 * v2)],
                [(1 / J2) * (- ((J1 + J2) / J1) * tau + c1 * (J2 / J1) * v1 - c2 * ((J1 + J2) / J1) * v2)]])

# Show the result
f

Matrix([
[                        v1],
[       tau - v1/10 + v2/10],
[-33*tau + v1/10 - 33*v2/10]])

Linearization:

In [45]:
q1e = 0.
v1e = 0.
v2e = 0.
taue = 0.

A_num = sym.lambdify((q1, v1, v2, tau), f.jacobian([q1, v1, v2]))
B_num = sym.lambdify((q1, v1, v2, tau), f.jacobian([tau]))
A = A_num(q1e, v1e, v2e, taue).astype(float)
B = B_num(q1e, v1e, v2e, taue).astype(float)

In [46]:
print(A)

[[ 0.   1.   0. ]
 [ 0.  -0.1  0.1]
 [ 0.   0.1 -3.3]]


In [47]:
print(B)

[[  0.]
 [  1.]
 [-33.]]


Controllability matrix:

In [48]:
W = np.block([[B, A @ B, A @ A @ B]])

In [49]:
print(W)

[[   0.      1.     -3.4 ]
 [   1.     -3.4    11.24]
 [ -33.    109.   -360.04]]


Rank of controllability matrix:

In [50]:
np.linalg.matrix_rank(W)

2

Number of states:

In [51]:
A.shape[0]

3

What happens if you try to do eigenvalue placement?

In [52]:
fbk = signal.place_poles(A, B, [-1., -2., -3.])
K = fbk.gain_matrix

Did it work?

In [53]:
print(linalg.eigvals(A - B @ K))

[-2921643.813426  +0.j       -7.84997703+0.j  2921648.80955044+0.j]


# What if there is gravity (i.e., non-zero ground pitch)?

Parameters:

In [54]:
# distance from platform axis to wheel axis
l = 1.

# radius of wheel
rw = 0.5

# mass of wheel
mw = 0.25

# moi of wheel
Jw = 0.5 * mw * rw**2

# mass of platform
mp = 12. * (1. - mw * l**2) / (3.**2 + 2.**2)

# moment of inertia of platform
Jp = (1. / 12.) * mp * (3.**2 + 2.**2)

# gravity
g = 9.81

Equations of motion:

$$
\begin{aligned}
J_1 \ddot{q}_1 &= \tau - m_wgl\sin(q_1) \\
J_2 \dot{v}_2 &= - \left( \dfrac{J_1 + J_2}{J_1} \right) \tau + \left( \dfrac{J_2}{J_1} \right) m_wgl\sin(q_1)
\end{aligned}
$$

where

$$
J_1 = J_p + m_wl^2
\qquad
\qquad
J_2 = J_w
$$

In [55]:
# Convert floats to rationals
J1 = sym.nsimplify(Jp + mw * l**2)
J2 = sym.nsimplify(Jw)
mw = sym.nsimplify(mw)
g = sym.nsimplify(g)
l = sym.nsimplify(l)

# Create symbolic variables
q1, v1, v2, tau = sym.symbols('q1, v1, v2, tau', real=True)

# Define equations of motion
f = sym.Matrix([[v1], [(1 / J1) * (tau - mw * g * l * sym.sin(q1))], [(1 / J2) * (- ((J1 + J2) / J1) * tau + (J2 / J1) * mw * g * l * sym.sin(q1))]])

# Show the result
f

Matrix([
[                       v1],
[    tau - 981*sin(q1)/400],
[-33*tau + 981*sin(q1)/400]])

Linearization:

Nnote that $\tau=0$ at any equilibrium point — check this yourself! — and so there are exact two choices for $q_1$:

$$q_1 = 0 \qquad\text{or}\qquad q_1 = \pi$$

In [56]:
q1e = np.pi
v1e = 0.
v2e = 0.
taue = 0.

A_num = sym.lambdify((q1, v1, v2, tau), f.jacobian([q1, v1, v2]))
B_num = sym.lambdify((q1, v1, v2, tau), f.jacobian([tau]))
A = A_num(q1e, v1e, v2e, taue).astype(float)
B = B_num(q1e, v1e, v2e, taue).astype(float)

In [57]:
print(A)

[[ 0.      1.      0.    ]
 [ 2.4525  0.      0.    ]
 [-2.4525  0.      0.    ]]


In [58]:
print(B)

[[  0.]
 [  1.]
 [-33.]]


Controllability matrix:

In [59]:
W = np.block([[B, A @ B, A @ A @ B]])

In [60]:
print(W)

[[  0.       1.       0.    ]
 [  1.       0.       2.4525]
 [-33.       0.      -2.4525]]


Rank of controllability matrix:

In [61]:
np.linalg.matrix_rank(W)

3

Number of states:

In [62]:
A.shape[0]

3

What happens if you try to do eigenvalue placement?

In [63]:
fbk = signal.place_poles(A, B, [-1., -2., -3.])
K = fbk.gain_matrix

Did it work?

In [64]:
print(linalg.eigvals(A - B @ K))

[-3.+0.j -2.+0.j -1.+0.j]


Print $K$ for copy/paste into a controller:

In [65]:
print(K.tolist())

[[13.452500000000049, 8.522935779816546, 0.07645259938837953]]


Do you think all those significant digits really matter?

In [67]:
K = K.round(decimals=4)
print(K.tolist())

[[13.4525, 8.5229, 0.0765]]


What happened to the eigenvalues?

In [68]:
print(linalg.eigvals(A - B @ K))

[-2.99464272+0.j -2.00269729+0.j -1.00105999+0.j]


# Questions to consider

* What if you want to control the *angle* of the wheel and not just the angular velocity, with gravity?
* What if there is only damping at one joint and not the other?
* What if there is both gravity and damping?
* What if you change the physical parameters (masses, moments of inertia, etc.)?
* Etc.